# 0.0. Importações

## 0.1. Bibliotecas

In [104]:
import ast
import nltk
import sklearn
import numpy as np
import pandas as pd
import streamlit as st

from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## 0.2. Dataset

In [72]:
df_raw = pd.read_csv('../data/amazon.csv')

## 0.3. Funções

In [73]:
# Função stemming
def stem(text):
  y=[]
  for i in text.split():
    y.append(parser_ps.stem(i))
  return " ".join(y)

# Função para o sistema de recomendação
def sistema_recomendacao(movie):
  index = df7[df7['product_name'] == movie].index[0]
  distances = sorted(list(enumerate(similaridades[index])), reverse=True, key=lambda x:x[1])

  for i in distances[1:6]:
    print(df7.iloc[i[0]].product_name)

# 1.0. Descrição dos Dados

In [74]:
df1 = df_raw.copy()

## 1.1. Shape

In [75]:
print(f'Number of rows: {df1.shape[0]}')
print(f'Number of cols: {df1.shape[1]}')

Number of rows: 1465
Number of cols: 16


## 1.2. Types

In [76]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1465 non-null   object
 1   product_name         1465 non-null   object
 2   category             1465 non-null   object
 3   discounted_price     1465 non-null   object
 4   actual_price         1465 non-null   object
 5   discount_percentage  1465 non-null   object
 6   rating               1465 non-null   object
 7   rating_count         1463 non-null   object
 8   about_product        1465 non-null   object
 9   user_id              1465 non-null   object
 10  user_name            1465 non-null   object
 11  review_id            1465 non-null   object
 12  review_title         1465 non-null   object
 13  review_content       1465 non-null   object
 14  img_link             1465 non-null   object
 15  product_link         1465 non-null   object
dtypes: obj

# 2.0. Filtros

In [77]:
df2 = df1.copy()

## 2.1. Colunas

In [78]:
df2 = df2[['product_id', 'product_name', 'category', 'about_product', 'review_title', 'review_content']]

# 3.0. Análise Exploratória de Dados

In [79]:
df3 = df2.copy()

## 3.1. Check NA

In [80]:
df3.isnull().sum()

product_id        0
product_name      0
category          0
about_product     0
review_title      0
review_content    0
dtype: int64

## 3.2. Shape

In [81]:
df3.shape

(1465, 6)

## 3.3. Tirar duplicados

In [82]:
df3 = df3.drop_duplicates()

In [83]:
df3.duplicated().sum()

0

# 4.0. Processamento de Texto

In [84]:
df4 = df3.copy()

In [85]:
df4.head(1)

,product_id,product_name,category,about_product,review_title,review_content
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,High Compatibility : Compatible With iPhone 12...,"Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...


## 4.1. Limpeza dos Dados de Texto

In [86]:
df4['category'] = df4['category'].apply(lambda x:x.split())
df4['about_product'] = df4['about_product'].apply(lambda x:x.split())
df4['review_title'] = df4['review_title'].apply(lambda x:x.split())
df4['review_content'] = df4['review_content'].apply(lambda x:x.split())

In [87]:
df4.head(1)

,product_id,product_name,category,about_product,review_title,review_content
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,[Computers&Accessories|Accessories&Peripherals...,"[High, Compatibility, :, Compatible, With, iPh...","[Satisfied,Charging, is, really, fast,Value, f...","[Looks, durable, Charging, is, fine, tooNo, co..."


## 4.2. Preparando os Dados para Vetorização em Outro Espaço Vetorial

In [88]:
df4['tags'] = df4['category'] + \
              df4['about_product'] + \
              df4['review_title'] + \
              df4['review_content']

In [89]:
# selecionar features que vão ser usadas
df4 = df4[['product_id', 'product_name', 'tags']]
# simplificar o vetor
df4['tags'] = df4['tags'].apply(lambda x:" ".join(x))
# colocar tudo em minúsculo
df4['tags'] = df4['tags'].apply(lambda x:x.lower())

In [90]:
df4.head(1)

,product_id,product_name,tags
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,computers&accessories|accessories&peripherals|...


# 5.0. Parse e Vetorização

In [91]:
df5 = df4.copy()

In [92]:
# criamos o parser
parser_ps = PorterStemmer()

## 5.1. Stemming

In [93]:
df5['tags'] = df5['tags'].apply(stem)

In [94]:
df5.head()

,product_id,product_name,tags
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,computers&accessories|accessories&peripherals|...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,computers&accessories|accessories&peripherals|...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,computers&accessories|accessories&peripherals|...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,computers&accessories|accessories&peripherals|...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,computers&accessories|accessories&peripherals|...


## 5.2. Vetorizador

In [95]:
atributos = 5000
cv = CountVectorizer(max_features=atributos, stop_words='english')

In [96]:
vectors = cv.fit_transform(df5['tags']).toarray()

In [97]:
len(cv.get_feature_names_out())

5000

In [98]:
type(vectors)

numpy.ndarray

In [99]:
vectors[3]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

# 6.0. Cálculo de Distância dos vetores

In [100]:
df6 = df5.copy()

In [101]:
similaridades = cosine_similarity(vectors)

# 7.0. Sistema de Recomendação

In [102]:
df7 = df6.copy()

In [103]:
sistema_recomendacao(df7['product_name'][3])

boAt Deuce USB 300 2 in 1 Type-C & Micro USB Stress Resistant, Sturdy Cable with 3A Fast Charging & 480mbps Data Transmission, 10000+ Bends Lifespan and Extended 1.5m Length(Mercurial Black)
boAt Rugged V3 Braided Micro USB Cable (Pearl White)
boAt Rugged v3 Extra Tough Unbreakable Braided Micro USB Cable 1.5 Meter (Black)
Gilary Multi Charging Cable, 3 in 1 Nylon Braided Fast Charging Cable for iPhone Micro USB Type C Mobile Phone | Colour May Vary |
Boat A 350 Type C Cable 1.5m(Jet Black)
